In [1]:
!pip install -qU langgraph langchain langchain-huggingface langchain-community duckduckgo-search python-dotenv pypdf faiss-cpu sentence-transformers

In [2]:
import os
import getpass

def get_mandatory_env(var_name, prompt):
    val = os.environ.get(var_name)
    if not val:
        val = getpass.getpass(prompt)
        os.environ[var_name] = val
    if not val:
        raise ValueError(f"{var_name} is MANDATORY. Please provide it.")
    return val

# 1. Hugging Face Token 
get_mandatory_env("HUGGINGFACEHUB_API_TOKEN", "Enter Hugging Face API Token: ")

# 2. LangSmith Configuration 
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Llama-Tutor-Agent"
get_mandatory_env("LANGCHAIN_API_KEY", "Enter LangSmith API Key: ")

print("Environment Configured Successfully.")

Enter Hugging Face API Token:  ········
Enter LangSmith API Key:  ········


Environment Configured Successfully.


In [13]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace, HuggingFaceEmbeddings
from langchain_community.tools import DuckDuckGoSearchRun

# MODEL CONFIGURATION
MODEL_REPO = "Qwen/Qwen2.5-72B-Instruct" 

# 1. Initialize Endpoint
llm_engine = HuggingFaceEndpoint(
    repo_id=MODEL_REPO,
    task="text-generation",
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.1,
    streaming=False, 
)

# 2. Wrap in Chat Interface 
llm = ChatHuggingFace(llm=llm_engine)

# 3. Initialize Embeddings & Search
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
web_search = DuckDuckGoSearchRun()

# 4. Define Checkpoints
CHECKPOINTS = {
    1: {"topic": "Transformer Architecture", "objective": "Explain the Encoder-Decoder structure and Self-Attention mechanism."},
    2: {"topic": "Backpropagation", "objective": "Detail the chain rule and how gradients update weights in a neural network."},
    3: {"topic": "RAG Systems", "objective": "Explain Retrieval-Augmented Generation, vector databases, and semantic search."},
    4: {"topic": "Generative Adversarial Networks", "objective": "Explain the Generator vs Discriminator dynamic and training challenges"},
    5: {"topic": "Convolutional Neural Networks", "objective": "Explain Filters, Pooling layers, and their application in image recognition."}
}

print(f"Model Ready: {MODEL_REPO}")

Model Ready: Qwen/Qwen2.5-72B-Instruct


In [14]:
# Cell 4: User Interaction 
import os

DEFAULT_FILE = "notes.pdf" 

# 1. Select Checkpoint
print("SELECT A LEARNING CHECKPOINT")
if 'CHECKPOINTS' not in globals():
    print("Error: CHECKPOINTS dict missing. Please Run Cell 3 first.")
else:
    for key, val in CHECKPOINTS.items():
        print(f"{key}. {val['topic']}")

    while True:
        try:
            choice_input = input("Enter choice (1-5): ")
            choice = int(choice_input)
            if choice in CHECKPOINTS:
                selected_checkpoint = CHECKPOINTS[choice]
                print(f"Selected: {selected_checkpoint['topic']}")
                break
            print("Invalid choice.")
        except ValueError:
            print("Please enter a number.")

    # 2. Auto-Detect Document
    if os.path.exists(DEFAULT_FILE):
        print(f"Auto-detected file: {DEFAULT_FILE}")
        pdf_path = DEFAULT_FILE
    else:
        # Fallback if file isn't there
        user_input = input(f"File '{DEFAULT_FILE}' not found. Enter filename manually: ")
        pdf_path = user_input if user_input else None

    if pdf_path and not os.path.exists(pdf_path):
        print(f"Warning: File '{pdf_path}' not found. Agent will rely on Web Search.")
        pdf_path = None

SELECT A LEARNING CHECKPOINT
1. Transformer Architecture
2. Backpropagation
3. RAG Systems
4. Generative Adversarial Networks
5. Convolutional Neural Networks


Enter choice (1-5):  3


Selected: RAG Systems
Auto-detected file: notes.pdf


In [15]:
from typing import TypedDict, Optional

class AgentState(TypedDict):
    topic: str
    objective: str
    file_path: Optional[str]
    
    # Content Storage
    doc_context: str
    web_context: str
    
    # Flags & Counters
    needs_web_search: bool
    retries: int
    
    # Output & Scoring
    final_essay: str
    relevance_score: int
    validation_reasoning: str
    
    # Best Effort Memory
    best_essay: str
    best_score: int

In [16]:
# Cell 6: Document Processing Node 
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.messages import SystemMessage, HumanMessage

def check_user_doc(state: AgentState):
    print("CHECKING USER DOCUMENT")
    file_path = state["file_path"]
    topic = state["topic"]
    
    # 1. Basic Check
    if not file_path:
        return {"doc_context": "", "needs_web_search": True}
    
    try:
        loader = PyPDFLoader(file_path)
        docs = loader.load()
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = splitter.split_documents(docs)
        
        if not splits:
            return {"doc_context": "", "needs_web_search": True}
        
        # 2. Vector Search
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
        retrieved_docs = retriever.invoke(topic)
        context_text = "\n\n".join([d.page_content for d in retrieved_docs])
        
        # 3. LLM GRADED CHECK
        grader_prompt = f"""
        You are a Relevance Grader. 
        Topic: {topic}
        Retrieved Text: {context_text[:2000]}
        
        Does the retrieved text contain a DETAILED explanation of the Topic?
        If it only contains headers, unrelated topics, or placeholders like "This chapter is blank", answer NO.
        
        Answer only YES or NO.
        """
        response = llm.invoke([HumanMessage(content=grader_prompt)])
        grade = response.content.strip().upper()
        
        if "NO" in grade or len(context_text) < 200:
            print(f"Doc content found ({len(context_text)} chars) but graded INCOMPLETE. Enabling Web Search.")
            return {"doc_context": context_text, "needs_web_search": True}
        
        print(f"Found relevant content in user doc. (Grader: {grade})")
        return {"doc_context": context_text, "needs_web_search": False}
        
    except Exception as e:
        print(f"Error reading doc: {e}")
        return {"doc_context": "", "needs_web_search": True}

In [17]:
def perform_web_search(state: AgentState):
    print("PERFORMING WEB SEARCH")
    
    if not state.get("needs_web_search"):
        print("Skipping web search (User Doc sufficient).")
        return {"web_context": ""}
        
    topic = state["topic"]
    objective = state["objective"]
    
    query = f"{topic} {objective} detailed technical explanation"
    try:
        search_results = web_search.invoke(query)
        print("Web search completed.")
        return {"web_context": search_results}
    except Exception as e:
        print(f"Search failed: {e}")
        return {"web_context": ""}

In [18]:
# Cell 8: Generation Node
from langchain_core.messages import SystemMessage, HumanMessage

def generate_essay(state: AgentState):
    print("GENERATING SUMMARY")
    
    doc_text = state.get("doc_context", "")
    web_text = state.get("web_context", "")
    topic = state["topic"]
    objective = state["objective"]
    
    system_instruction ="""You are an expert AI Tutor. Write a CONCISE Technical Summary (approx 3 paragraphs).
    
    RULES:
    1. Keep it under 400 words.
    2. STRICT CITATIONS: End sentences with [Source: User Doc] or [Source: Web Search].
    3. If User Doc is irrelevant/blank, rely fully on Web Search.
    """
    
    user_content = f"""
    TOPIC: {topic}
    OBJECTIVE: {objective}
    
    CONTEXT FROM USER DOC:
    {doc_text[:3000]}
    
    CONTEXT FROM WEB SEARCH:
    {web_text[:3000]}
    
    Write the summary now.
    """
    
    messages = [
        SystemMessage(content=system_instruction),
        HumanMessage(content=user_content)
    ]
    
    try:
        response_msg = llm.invoke(messages)
        return {"final_essay": response_msg.content}
    except Exception as e:
        print(f"\n GENERATION ERROR: {e}\n")
        return {"final_essay": f"Error generating essay: {str(e)}"}

In [19]:
# Cell 9: Validation Node
import re
import json
from langchain_core.messages import HumanMessage

def validate_output(state: AgentState):
    print("VALIDATING OUTPUT")
    essay = state["final_essay"]
    objective = state["objective"]
    current_retries = state.get("retries", 0)
    best_score = state.get("best_score", 0)
    best_essay = state.get("best_essay", "")
    
    if "Error generating essay" in essay:
        score = 1
        reason = "Generation Failed"
    else:
        # UPDATED CHECKLIST
        prompt = f"""
        Grade the following summary on a scale of 1-5 using this EXACT checklist:
        
        +1 Point: Is it relevant to "{objective}"?
        +1 Point: Is it clear and concise?
        +1 Point: Does it contain citations like [Source: User Doc] or [Source: Web Search]?
        +1 Point: Is the content technically accurate?
        +1 Point: Is the structure clear?
        
        Total Score = Sum of points. (Max 5).
        
        Return ONLY valid JSON: {{"score": int, "reasoning": "string"}}
        
        SUMMARY START:
        {essay[:1500]}...
        """
        try:
            response = llm.invoke([HumanMessage(content=prompt)])
            json_match = re.search(r"\{.*\}", response.content, re.DOTALL)
            if json_match:
                data = json.loads(json_match.group(0))
                score = data.get("score", 0)
                reason = data.get("reasoning", "No reasoning")
            else:
                score = 4 
                reason = "JSON Parse Error, but content generated."
        except Exception as e:
            print(f"Validation failed: {e}")
            score = 3
            reason = "Validation Exception"

    print(f"Attempt {current_retries + 1} Score: {score}/5 | Reason: {reason}")

    if score > best_score:
        print(f"New High Score! ({score})")
        best_score = score
        best_essay = essay

    return {
        "relevance_score": score, 
        "validation_reasoning": reason,
        "retries": current_retries + 1,
        "best_score": best_score,
        "best_essay": best_essay
    }

In [20]:
import time

def check_retry(state: AgentState):
    score = state["relevance_score"]
    retries = state["retries"]
    
    # Success
    if score >= 4:
        print("Score meets threshold. Finishing.")
        return "success"
    
    # Max Retries
    if retries >= 5:
        print("Max retries reached. Accepting best effort.")
        return "max_retries"
    
    # Retry with Delay
    print("Score too low. Retrying in 2 seconds...")
    time.sleep(2)
    return "retry"

def finalize_submission(state: AgentState):
    print("FINALIZING SUBMISSION")
    return {
        "final_essay": state["best_essay"], 
        "relevance_score": state["best_score"]
    }

In [21]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("check_user_doc", check_user_doc)
workflow.add_node("perform_web_search", perform_web_search)
workflow.add_node("generate_essay", generate_essay)
workflow.add_node("validate_output", validate_output)
workflow.add_node("finalize_submission", finalize_submission)

# Set Entry Point
workflow.set_entry_point("check_user_doc")

# Standard Edges
workflow.add_edge("check_user_doc", "perform_web_search")
workflow.add_edge("perform_web_search", "generate_essay")
workflow.add_edge("generate_essay", "validate_output")

# Conditional Edges
workflow.add_conditional_edges(
    "validate_output",
    check_retry,
    {
        "success": "finalize_submission",
        "max_retries": "finalize_submission",
        "retry": "generate_essay"
    }
)

workflow.add_edge("finalize_submission", END)

app = workflow.compile()
print("Robust Learning Graph Compiled.")

Robust Learning Graph Compiled.


In [22]:
# Cell 11: Execute Agent
inputs = {
    "topic": selected_checkpoint["topic"],
    "objective": selected_checkpoint["objective"],
    "file_path": pdf_path,
    "needs_web_search": False, 
    "doc_context": "",
    "web_context": "",
    # Initialize counters
    "retries": 0,
    "best_score": 0,
    "best_essay": ""
}

print(f"Starting Robust Agent for: {inputs['topic']}")

# Run the agent ONLY ONCE
result = app.invoke(inputs)

print("\n" + "="*50)
print(f"FINAL OUTPUT (Score: {result['relevance_score']}/5)")
print("="*50)
print(result["final_essay"])

Starting Robust Agent for: RAG Systems
CHECKING USER DOCUMENT
Doc content found (3129 chars) but graded INCOMPLETE. Enabling Web Search.
PERFORMING WEB SEARCH
Web search completed.
GENERATING SUMMARY
VALIDATING OUTPUT
Attempt 1 Score: 5/5 | Reason: The summary is relevant to the topic, explaining Retrieval-Augmented Generation, vector databases, and semantic search. It is clear and concise, providing examples and explanations that are easy to understand. The content includes citations from both user documentation and web searches. The technical accuracy is maintained throughout the summary, and the structure is clear and logical.
New High Score! (5)
Score meets threshold. Finishing.
FINALIZING SUBMISSION

FINAL OUTPUT (Score: 5/5)
Retrieval-Augmented Generation (RAG) is a technique designed to enhance the output of large language models (LLMs) by integrating an external knowledge base. Unlike traditional LLMs that rely solely on their pre-trained data, RAG systems can access and utiliz